In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("superstore_data.csv")

# Fixing IDs
max_id = df["Id"].max()
df.loc[df["Id"] == 0, "Id"] = max_id + 1

# Missing value imputation
# Check for missing values
missing = df.isnull().sum()
if missing[missing > 0].empty:
    print("There are no missing values in the dataset.")
else:
    print("Missing values found and will be filled.")
    
df = df.drop_duplicates()

# Fill with mean values
df = df.fillna(df.mean(numeric_only=True).astype(int))

# Data filtering
df.loc[df['Year_Birth'] < 1960, 'Year_Birth'] = 1960
df = df[df['Income'] <= 100000]

# Outliers
df.loc[df['MntWines'] < 15, 'MntWines'] = 0
df.loc[df['MntWines'] > 500, 'MntWines'] = 500
df.loc[df['MntFruits'] > 75, 'MntFruits'] = 75
df.loc[df['MntMeatProducts'] < 10, 'MntMeatProducts'] = 0
df.loc[df['MntMeatProducts'] > 500, 'MntMeatProducts'] = 500
df.loc[df['MntFishProducts'] > 100, 'MntFishProducts'] = 100
df.loc[df['MntFishProducts'] < 10, 'MntFishProducts'] = 0
df.loc[df['MntSweetProducts'] > 65, 'MntSweetProducts'] = 65
df.loc[df['MntGoldProds'] > 125, 'MntGoldProds'] = 125

# Feature Engineering
df['Age'] = 2026 - df['Year_Birth'] 
df['Total_Children'] = df['Kidhome'] + df['Teenhome']

# Determine the number of years the client has been registered with the company.
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)
df['Customer_For_Years'] = 2026 - df['Dt_Customer'].dt.year

df = df.drop(columns = ['Year_Birth'])
df = df.drop(columns = ['Dt_Customer'])

# Binning
df['Education'] = df['Education'].replace({'2n Cycle': 'Master'})

df['Marital_Status'] = df['Marital_Status'].replace({
    'Alone': 'Single', 
    'Absurd': 'Single', 
    'YOLO': 'Single',
    'Widow': 'Single',
    'Together': 'Married'
})
df = df.drop_duplicates()

df.to_csv("superstore_data_cleaned.csv", index=False)
print("The file has been successfully saved with the specified encoding")

Missing values found and will be filled.
The file has been successfully saved with the specified encoding


C:\Users\user\AppData\Local\Temp\ipykernel_14088\922591687.py:46: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)


In [8]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from dotenv import load_dotenv
from datetime import datetime

load_dotenv(override=True)

url_object = URL.create(
    drivername="postgresql+psycopg2",
    username=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    database=os.getenv("DB_NAME")
)

engine = create_engine(url_object, connect_args={'options': '-c client_encoding=utf8'})

csv_file_path = "superstore_data_cleaned.csv" 

df = pd.read_csv(csv_file_path, encoding='utf-8')

print(f" File successfully read! Rows: {len(df)}")

def generate_random_timestamps(num_records):
    start_date = datetime(2026, 1, 1)
    end_date = datetime(2026, 12, 31, 23, 59, 59)

    start_timestamp = pd.Timestamp(start_date).value // 10**9
    end_timestamp = pd.Timestamp(end_date).value // 10**9
    
    random_timestamps = np.random.randint(start_timestamp, end_timestamp, num_records)
    return pd.to_datetime(random_timestamps, unit='s')

df['timestamp'] = generate_random_timestamps(len(df))

table_name = "superstore_data_cleaned"

try:
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists='replace',
        index=False,
        method='multi',
        chunksize=1000 
    )
    print("Data successfully loaded into the database")
except Exception as e:
    print(f"Critical error during write: {e}")

 File successfully read! Rows: 2227
Data successfully loaded into the database
